# Multi i-TED: CRT study
## First: No energy filters and no position

- Study of CRT using different algorithms:
    - First event (SKEW=0)
    - Average of the first n events (DEGREE=0)
    - Energy-weighted average of the first n events (DEGREE=1)
    - Second power of the enery-weighted average of the first n events (DEGREE=2)
    - Third power of the enery-weighted average of the first n events (DEGREE=3)
- The measurement used in a Na22 measurement between the absorber and scatterer planes of iTED:
    - It doesn't include any filter around the 511keV energy peak
    - The measurement was performed for orientation checks, not timing studies
    - Only 1 crystal in the absorber plane is used
- The calculation of the timing:
    - The timing weighted is calculated as energy^DEGREE
    - In case energy^DEGREE causes an error (`floating point exception`), then it is considered 0 and not taking into account for the average
    - In case the sum of the energy^DEGREE is 0, then the time will be 0, as all weights were too low

In [1]:
pkg_ver = lambda pkg: "{:<20}{:}".format(pkg.__name__,pkg.__version__)

# ROOT
import uproot
print(pkg_ver(uproot))
import ROOT

# Machine Learning
import sklearn
print(pkg_ver(sklearn))
import torch
print(pkg_ver(torch))

# Data science
import scipy
print(pkg_ver(scipy))
import numpy
print(pkg_ver(numpy))
import pandas
print(pkg_ver(pandas))

# Visualizations
import matplotlib
print(pkg_ver(matplotlib))
import matplotlib.pyplot as plt

import tqdm
print(pkg_ver(tqdm))

uproot              4.3.5
Welcome to JupyROOT 6.28/02
sklearn             1.2.2
torch               2.0.0
scipy               1.10.1
numpy               1.23.5
pandas              1.5.3
matplotlib          3.7.1
tqdm                4.62.3


In [2]:
%jsroot

In [3]:
file = lambda skew, degree: f"../../data/Multi_iTED_Timing/Cs137_iTEDD_A1_D.2023_03_31_T.16_45_32_C.itedABCD_lab_custom_2023.02.22_4.0v_887_300s_serie1_3_CW100_{skew}_{degree}.root"

In [4]:
skew = [i+1 for i in range(9)]
degree = [0,1,2,3]

spectra = pandas.DataFrame(index = skew, columns = degree)

In [5]:
for i_skew in skew:
    for i_degree in degree:
        f_root = ROOT.TFile.Open(file(i_skew,i_degree))
        t_root = f_root.COINCIDENCES
        
        canvas = ROOT.TCanvas()
        canvas.cd()
        
        t_root.Draw("Delta_t[15]-Delta_t[16]","deposited_energy[15]>0.0&&deposited_energy[16]>0.0","")
        h_root = ROOT.gPad.GetPrimitive("htemp")
        h_root.Draw()
                
        latex = ROOT.TLatex()
        latex.SetNDC()
        latex.SetTextSize(0.03)
        latex.DrawText(0.25, 0.75, f"Skew:{i_skew}")
        latex.DrawText(0.25, 0.7, f"Degree:{i_degree}")        
        
        spectra[i_degree][i_skew] = {
            "histo":canvas,
            "mean":h_root.GetMean(),
            "std":h_root.GetStdDev(),
            "skew":h_root.GetSkewness(),
            "kurt":h_root.GetKurtosis(),
        }

## Analysis

### Standard deviation

In [6]:
spectra.applymap(lambda x: x["std"]).style.background_gradient(cmap ='YlOrRd',axis=None)

,0,1,2,3
1,3.573781,3.573781,3.573781,3.573781
2,3.800010,3.883709,3.814320,2.668088
3,4.095560,4.109220,3.970378,3.276534
4,4.296068,4.263600,4.068106,3.775289
5,4.476399,4.365311,4.156889,3.955185
6,4.733703,4.458765,4.268667,4.040145
7,4.848139,4.541241,4.284617,4.074858
8,4.995210,4.620758,4.343224,4.138087
9,5.065218,4.713147,4.409036,4.183250


### Mean value, skewness, and kurtosis

In [7]:
spectra.applymap(lambda x: x["mean"]).style.background_gradient(cmap ='YlOrRd',axis=None)

,0,1,2,3
1,0.070741,0.070741,0.070741,0.070741
2,0.197484,0.019415,-0.014096,0.052955
3,0.450172,0.123749,0.072265,0.113296
4,0.692114,0.320551,0.181902,0.202839
5,0.871618,0.520607,0.316192,0.264931
6,1.093534,0.753547,0.413800,0.292875
7,1.380429,0.955509,0.554942,0.342473
8,1.670457,1.203847,0.653008,0.362433
9,1.971788,1.421076,0.741232,0.373948


In [8]:
spectra.applymap(lambda x: x["skew"]).style.background_gradient(cmap ='YlOrRd',axis=None)

,0,1,2,3
1,-0.267641,-0.267641,-0.267641,-0.267641
2,-0.345871,-0.312437,-0.264907,-0.316010
3,-0.383286,-0.334098,-0.314305,-0.335093
4,-0.458243,-0.364846,-0.354381,-0.356456
5,-0.547817,-0.427416,-0.376457,-0.361717
6,-0.582732,-0.489290,-0.417827,-0.378565
7,-0.618411,-0.560143,-0.439715,-0.377715
8,-0.650658,-0.601400,-0.474658,-0.378327
9,-0.695256,-0.664820,-0.496359,-0.382950


In [9]:
spectra.applymap(lambda x: x["kurt"]).style.background_gradient(cmap ='YlOrRd',axis=None)

,0,1,2,3
1,0.217069,0.217069,0.217069,0.217069
2,-0.048478,-0.056858,-0.046332,2.984983
3,-0.153403,-0.218629,-0.118687,1.052188
4,-0.222336,-0.277633,-0.186515,0.172933
5,-0.337635,-0.313119,-0.198272,-0.023699
6,-0.443332,-0.313817,-0.215043,-0.070166
7,-0.480953,-0.294315,-0.216357,-0.086369
8,-0.492722,-0.295492,-0.221222,-0.112338
9,-0.482865,-0.254810,-0.212794,-0.105455


## Plots

Some notes:
- For 3 cases (DEGREE=3,SKEW=[2,3,4]), there is a high number of events with 0 time diference, it is due to the limitation of not being able to calculate very small numbers with c++'s pow
- The negative values of skewness, meaning that the values tend to be right-leaning, indicate that the events happen first in [15] and then [16]

Improvements:
- It is entirely possible that with energy filters, the outliers are suppressed and the shape of the dstribution changes
- In order to draw conclusions, a study with a controlled Na22 measurement needs to be performed

In [10]:
for i_skew in skew:
    for i_degree in degree:
        canvas.cd()
        
        spectra[i_degree][i_skew]["histo"].Draw()
        
        canvas.Draw()